In [128]:
#1.3 ellipsoid
# Let M be an ellipsoid in R3, (ax)^{2}+(by)^{2}+z^{2}=d
# x = 1/a*cos(p)*sin(t)*e1 + 1/b*sin(p)*sin(t)*e2 + cos(t)*e3, t, p in R

from gc_utils import *
alg = Algebra(3)
locals().update(alg.blades)

c2e = lambda p, t: 4*np.cos(p)*np.sin(t)*e1 + 5*np.sin(p)*np.sin(t)*e2 + np.cos(t)*e3
e_p = lambda p, t: -4*np.sin(p)*np.sin(t)*e1 + 5*np.cos(p)*np.sin(t)*e2
e_t = lambda p, t: (4*np.cos(p)*np.cos(t)*e1) + (5*np.sin(p)*np.cos(t)*e2) - (np.sin(t)*e3)
def Ix(x):
    p = np.arctan((4/5)*x.e2/x.e1)
    t = np.arccos(x.e3)
    return e_p(p, t) ^ e_t(p, t)

# |p| < pi/2, 0 < t < pi for unique arctan & arccos
p, t = -1, 3
x = c2e(p, t)
I = Ix(x)
nIx = lambda x: normalize(Ix(x))
nI = nIx(x)

nI, nI ^ (3*e_t(p,t) + 5*e_p(p,t))

(1.0 𝐞₁₂ + -0.024 𝐞₁₃ + -0.0192 𝐞₂₃, 1.29e-15 𝐞₁₂₃)

In [129]:
#5.1
C, D, E = [random_multivector(alg) for _ in range(3)]
V1, V2 = [random_versor(2, alg) for _ in range(2)]
f = lambda x: V1.sw(inv(x)) + V2.sw(x**3)
A = lambda x: E*x**2 + C*x*D
a = lambda x: A(x).grade(1)
PA = lambda x: P(A(x), Ix(x))
pa = lambda x: P(a(x), Ix(x))
f_ = lambda a: lambda x: differential(f, x, a)

x, f(x), pa(x), f_(pa(x))(x)

(0.305 𝐞₁ + -0.594 𝐞₂ + -0.99 𝐞₃,
 -0.36 𝐞₁ + 0.0326 𝐞₂ + -1.7 𝐞₃,
 0.0418 𝐞₁ + 0.0452 𝐞₂ + -0.000279 𝐞₃,
 0.0858 𝐞₁ + 0.0447 𝐞₂ + 0.0329 𝐞₃)

In [130]:
#5.2 For tangent fields, the outermorphism can be computed in the embedded space
# the scalar product in the skew_symmetrizer acts as a projection into the tangent space
def outermorphism(f, field, alg, h=1e-6, frame=None, r_frame=None, Ix=None):
    def f_outer(x):
        nonlocal frame, r_frame
        if Ix:
            frame = blade_split(Ix(x), alg)
            r_frame = reciprocal(frame)
        A = field(x)
        wf = lambda vectors: wedge([f(v)(x) for v in vectors])
        outer = 0
        for r in A.grades:
            if r == 0:
                outer += A.grade(0)
                continue
            outer += skew_symmetrizer(wf, A.grade(r), alg, h, frame, r_frame)
        return outer
    return f_outer

outermorphism(f_, A, alg)(x), outermorphism(f_, A, alg, Ix=Ix)(x), outermorphism(f_, PA, alg)(x), outermorphism(f_, PA, alg, Ix=Ix)(x)

(0.162 + 0.241 𝐞₁ + -0.0937 𝐞₂ + 0.569 𝐞₃ + -6.97 𝐞₁₂ + 2.39 𝐞₁₃ + 9.14 𝐞₂₃ + -3.14 𝐞₁₂₃,
 0.162 + 0.0858 𝐞₁ + 0.0447 𝐞₂ + 0.033 𝐞₃ + -2.05 𝐞₁₂ + -4.6 𝐞₁₃ + -1.61 𝐞₂₃ + 5.54e-19 𝐞₁₂₃,
 0.162 + 0.0858 𝐞₁ + 0.0447 𝐞₂ + 0.0329 𝐞₃ + -2.04 𝐞₁₂ + -4.59 𝐞₁₃ + -1.61 𝐞₂₃,
 0.162 + 0.0858 𝐞₁ + 0.0447 𝐞₂ + 0.033 𝐞₃ + -2.05 𝐞₁₂ + -4.6 𝐞₁₃ + -1.61 𝐞₂₃ + 5.54e-19 𝐞₁₂₃)

In [131]:
#5.3
f_vec = lambda a: lambda x: differential(f, x, a)
f_ = lambda A: outermorphism(f_vec, A, alg, Ix=Ix)
I1 = f_(Ix)(x)
fnI = f_(nIx)(x)
norm(I1)/norm(I), norm(fnI)

(3.7812362482047757, 3.7812362482047766)

In [ ]:
# Jacobian
Jf = lambda x: norm(f_(nIx)(x))

In [133]:
#5.4
def adjoint_outermorphism(f, field, alg, h=1e-6, frame=None, r_frame=None, Ix=None):
    def f_outer(x):
        A = field(x)
        nonlocal frame, r_frame
        if Ix:
            frame = blade_split(Ix(x), alg)
            r_frame = reciprocal(frame)
        F = lambda vectors: wedge([f(v)(x) for v in vectors]).sp(A)
        outer = 0
        for r in A.grades:
            if r == 0:
                outer += A.grade(0)
                continue
            # Why np.zeros(r)? The derivative of a linear function is constant
            outer += simplicial_derivative(F, np.zeros(r), alg, h, frame, r_frame)
        return outer
    return f_outer

def adjoint_outermorphism_(f, A, alg, h=1e-6, frame=None, r_frame=None, Ix=None):
    def f_outer(x):
        nonlocal frame, r_frame
        if Ix:
            frame = blade_split(Ix(x), alg)
            r_frame = reciprocal(frame)
        F = lambda vectors: wedge([f(v)(x) for v in vectors]).sp(A)
        outer = 0
        for r in A.grades:
            if r == 0:
                outer += A.grade(0)
                continue
            # Why np.zeros(r)? The derivative of a linear function is constant
            outer += simplicial_derivative(F, np.zeros(r), alg, h, frame, r_frame)
        return outer
    return f_outer

adjoint_outermorphism(f_vec, A, alg, Ix=Ix)(x)

0.162 + -0.164 𝐞₁ + 0.386 𝐞₂ + -0.0124 𝐞₃ + 3.69 𝐞₁₂ + -0.0884 𝐞₁₃ + -0.071 𝐞₂₃

In [134]:
240/(2*10*200/1000*7.3), 100/ (np.pi*49/4), 130/100*5

(8.219178082191782, 2.598448050479924, 6.5)

In [135]:
#5.10a
f_(A)(x), f_(PA)(x), P(f_(A)(x), I1)

(0.162 + 0.0858 𝐞₁ + 0.0447 𝐞₂ + 0.033 𝐞₃ + -2.05 𝐞₁₂ + -4.6 𝐞₁₃ + -1.61 𝐞₂₃ + 5.54e-19 𝐞₁₂₃,
 0.162 + 0.0858 𝐞₁ + 0.0447 𝐞₂ + 0.033 𝐞₃ + -2.05 𝐞₁₂ + -4.6 𝐞₁₃ + -1.61 𝐞₂₃ + 5.54e-19 𝐞₁₂₃,
 0.162 + 0.0858 𝐞₁ + 0.0447 𝐞₂ + 0.033 𝐞₃ + -2.05 𝐞₁₂ + -4.6 𝐞₁₃ + -1.61 𝐞₂₃)

In [136]:
#5.10b
P_ = lambda A: lambda x: P(A(x), I)
P__ = lambda A: lambda x: P(A(x), I1)
fa = lambda A: adjoint_outermorphism(f_vec, A, alg, Ix=Ix)
fa(A)(x), fa(P__(A))(x), P_(fa(A))(x)

(0.162 + -0.164 𝐞₁ + 0.386 𝐞₂ + -0.0124 𝐞₃ + 3.69 𝐞₁₂ + -0.0884 𝐞₁₃ + -0.071 𝐞₂₃,
 0.162 + -0.164 𝐞₁ + 0.386 𝐞₂ + -0.0124 𝐞₃ + 3.69 𝐞₁₂ + -0.0884 𝐞₁₃ + -0.071 𝐞₂₃,
 0.162 + -0.164 𝐞₁ + 0.386 𝐞₂ + -0.0124 𝐞₃ + 3.69 𝐞₁₂ + -0.0884 𝐞₁₃ + -0.071 𝐞₂₃)

In [137]:
#5.12 outermorphism push forward tangent vectors
G = lambda x: (x+D)**2
F = lambda x: G(f(x))
differential(F, x, pa(x)), differential(G, f(x), f_(pa)(x))

(-0.0382 + 0.167 𝐞₁ + 0.0872 𝐞₂ + 0.0642 𝐞₃ + 0.0406 𝐞₁₂ + -0.0552 𝐞₁₃ + 0.106 𝐞₂₃ + 0.0313 𝐞₁₂₃,
 -0.0383 + 0.167 𝐞₁ + 0.0872 𝐞₂ + 0.0643 𝐞₃ + 0.0407 𝐞₁₂ + -0.0552 𝐞₁₃ + 0.106 𝐞₂₃ + 0.0314 𝐞₁₂₃)

In [138]:
#5.12c adjoint-outermorphism is the pullback/chain rule of derivatives
# without explicit define the inverse of f, we push forward the frame

def f_derivative_gen(x, f, alg, Ix, h=1e-6):
    f_vec = lambda a: lambda x: differential(f, x, a)
    f_ = lambda A: outermorphism(f_vec, A, alg, Ix=Ix)
    for v, r in frame_gen(x, alg, Ix=f_(Ix)):
        yield (r, lambda F: differential(F, f(x), v, h))

# Say f: M -> N
# "back" as the derivative acts on a field G on N
# Yet mimicking the derivative of G(f(x)) on M
def back_derivative_gen(x, f, alg, Ix, h=1e-6):
    f_vec = lambda a: lambda x: differential(f, x, a)
    f_ = lambda A: outermorphism(f_vec, A, alg, Ix=Ix)
    fa_ = lambda A: adjoint_outermorphism_(f_vec, A, alg, Ix=Ix)
    for v, r in frame_gen(x, alg, Ix=f_(Ix)):
        yield (fa_(r)(x), lambda F: differential(F, f(x), v, h))


fa_ = lambda A: adjoint_outermorphism_(f_vec, A, alg, Ix=Ix)
differential(F, x, pa(x)), sum((pa(x)|r) * o(G) for r, o in back_derivative_gen(x, f, alg, Ix))

(-0.0382 + 0.167 𝐞₁ + 0.0872 𝐞₂ + 0.0642 𝐞₃ + 0.0406 𝐞₁₂ + -0.0552 𝐞₁₃ + 0.106 𝐞₂₃ + 0.0313 𝐞₁₂₃,
 -0.0383 + 0.167 𝐞₁ + 0.0872 𝐞₂ + 0.0643 𝐞₃ + 0.0407 𝐞₁₂ + -0.0552 𝐞₁₃ + 0.106 𝐞₂₃ + 0.0314 𝐞₁₂₃)

In [139]:
#5.13
(
    derivative(F, x, alg, Ix=Ix), 
    sum(r * o(G) for r, o in back_derivative_gen(x, f, alg, Ix))
    )

(4.33 + 0.864 𝐞₁ + -5.48 𝐞₂ + 1.42 𝐞₃ + -2.03 𝐞₁₂ + -3.49 𝐞₁₃ + 3.53 𝐞₂₃ + 2.74 𝐞₁₂₃,
 4.33 + 0.863 𝐞₁ + -5.49 𝐞₂ + 1.42 𝐞₃ + -2.03 𝐞₁₂ + -3.49 𝐞₁₃ + 3.53 𝐞₂₃ + 2.74 𝐞₁₂₃)

In [140]:
def outermorphism_(f, field, alg, h=1e-6, frame=None, r_frame=None, Ix=None):
    def f_outer(x):
        nonlocal frame, r_frame
        if Ix:
            frame = blade_split(Ix(x), alg)
            r_frame = reciprocal(frame)
        wf = lambda vectors: wedge([f(v)(x) for v in vectors])
        outer = 0
        A = field if isinstance(field, MultiVector) else field(x)
        for r in A.grades:
            if r == 0:
                outer += A.grade(0)
                continue
            outer += skew_symmetrizer(wf, A.grade(r), alg, h, frame, r_frame)
        return outer
    return f_outer

In [141]:
#5.14 "forward" as the derivative acts on a field F on M
# Yet mimicking the derivative of F(inv(f)(x)) on N

def forward_derivative_gen(x, f, alg, Ix, h=1e-6):
    f_vec = lambda a: lambda x: differential(f, x, a)
    f_ = lambda A: outermorphism_(f_vec, A, alg, Ix=Ix)
    nIx = lambda x: normalize(Ix(x))
    nI = nIx(x)
    iI1 = inv(f_(nIx)(x))
    for v, r in frame_gen(x, alg, Ix=Ix):
        yield iI1 * f_(nI*r)(x), lambda F: differential(F, x, v, h)

iI1 = inv(f_(nIx)(x))
f__ = lambda A: outermorphism_(f_vec, A, alg, Ix=Ix)
(
    sum(r * o(G) for r, o in f_derivative_gen(x, f, alg, Ix)),
    sum(r * o(F) for r, o in forward_derivative_gen(x, f, alg, Ix)),
    sum(iI1 * f__(nI*r)(x) * o(F) for r, o in derivative_gen(x, alg, Ix=Ix))
    )

(3.9 + 0.615 𝐞₁ + -0.633 𝐞₂ + -1.91 𝐞₃ + 0.321 𝐞₁₂ + -0.193 𝐞₁₃ + 0.142 𝐞₂₃ + 2.47 𝐞₁₂₃,
 3.9 + 0.616 𝐞₁ + -0.633 𝐞₂ + -1.91 𝐞₃ + 0.321 𝐞₁₂ + -0.193 𝐞₁₃ + 0.141 𝐞₂₃ + 2.47 𝐞₁₂₃,
 3.9 + 0.616 𝐞₁ + -0.633 𝐞₂ + -1.91 𝐞₃ + 0.321 𝐞₁₂ + -0.193 𝐞₁₃ + 0.141 𝐞₂₃ + 2.47 𝐞₁₂₃)

In [142]:
#5.15a Without the inverse of f
# Pushing forward a differential is another linear transformation
b = random_r_blade(1, alg)
pb = P(b, I)
pb1 = f__(pb)(x)
differential(f__(PA(x)), x, pb, h=1e-2), sum((pb1 | r) * o(f__(PA(x))) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-2))

(-0.0714 𝐞₁ + -0.0224 𝐞₂ + -0.115 𝐞₃ + 2.49 𝐞₁₂ + 4.75 𝐞₁₃ + -1.36 𝐞₂₃ + 6.29e-17 𝐞₁₂₃,
 -1.18e-16 + -0.0708 𝐞₁ + -0.0209 𝐞₂ + -0.115 𝐞₃ + 2.44 𝐞₁₂ + 4.65 𝐞₁₃ + -1.46 𝐞₂₃ + 1.84e-18 𝐞₁₂₃)

In [143]:
#5.15b
differential(fa_(PA(x)), x, pb, h=1e-2), sum((pb1 | r) * o(fa_(PA(x))) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-2))

(0.00608 𝐞₁ + -0.0934 𝐞₂ + 0.0061 𝐞₃ + 2.4 𝐞₁₂ + -0.109 𝐞₁₃ + 0.0205 𝐞₂₃,
 -9.81e-17 + 0.00615 𝐞₁ + -0.0914 𝐞₂ + 0.00606 𝐞₃ + 2.36 𝐞₁₂ + -0.108 𝐞₁₃ + 0.0212 𝐞₂₃ + -2.7e-18 𝐞₁₂₃)

In [144]:
#5.16
(
    differential(f__(C^D), x, pb, h=1e-3), 
    (differential(f__(C), x, pb, h=1e-3)^f__(D)(x)) + (f__(C)(x)^differential(f__(D), x, pb, h=1e-3)),
    differential(fa_(C^D), x, pb, h=1e-3), 
    (differential(fa_(C), x, pb, h=1e-3)^fa_(D)(x)) + (fa_(C)(x)^differential(fa_(D), x, pb, h=1e-3)),
    )

(-1.68 𝐞₁ + -1.22 𝐞₂ + -1.75 𝐞₃ + -1.74 𝐞₁₂ + -2.26 𝐞₁₃ + 0.434 𝐞₂₃ + -2.56e-15 𝐞₁₂₃,
 -1.68 𝐞₁ + -1.22 𝐞₂ + -1.75 𝐞₃ + -1.74 𝐞₁₂ + -2.26 𝐞₁₃ + 0.434 𝐞₂₃ + 0.000347 𝐞₁₂₃,
 -0.0416 𝐞₁ + -2.28 𝐞₂ + 0.112 𝐞₃ + -5.43 𝐞₁₂ + 0.299 𝐞₁₃ + -0.115 𝐞₂₃,
 -0.0416 𝐞₁ + -2.28 𝐞₂ + 0.112 𝐞₃ + -5.43 𝐞₁₂ + 0.299 𝐞₁₃ + -0.115 𝐞₂₃ + 3.27e-05 𝐞₁₂₃)

In [145]:
#5.17a
Pb_ = lambda A: lambda x: differential(lambda a: P(A(x), Ix(a)), x, pb, h=1e-2)
Pb_1 = lambda A: lambda x: differential(lambda a: P(A(x), f_(Ix)(a)), x, pb, h=1e-2)
fb = lambda C: differential(f__(C), x, pb, h=1e-2)
C_ = lambda x: C
(
    fb(C),
    Pb_1(f__(C))(x)+ P(fb(C), I1),
    fb(P(C, I)) + f_(Pb_(C_))(x)
    )

(-0.817 𝐞₁ + -0.406 𝐞₂ + -1.16 𝐞₃ + -0.652 𝐞₁₂ + -1.23 𝐞₁₃ + 0.388 𝐞₂₃ + 8.09e-16 𝐞₁₂₃,
 1.11e-14 + -0.818 𝐞₁ + -0.405 𝐞₂ + -1.16 𝐞₃ + -0.652 𝐞₁₂ + -1.24 𝐞₁₃ + 0.388 𝐞₂₃,
 -0.817 𝐞₁ + -0.406 𝐞₂ + -1.16 𝐞₃ + -0.652 𝐞₁₂ + -1.23 𝐞₁₃ + 0.388 𝐞₂₃ + 8.09e-16 𝐞₁₂₃)

In [146]:
#5.17b
fb = lambda C: differential(fa_(C), x, pb, h=1e-2)

(
    fb(C),
    Pb_(fa_(C))(x)+ P(fb(C), I),
    fb(P(C, I1)) + fa_(Pb_1(C_)(x))(x)
    )

(-0.0517 𝐞₁ + -1.15 𝐞₂ + 0.0548 𝐞₃ + -2.89 𝐞₁₂ + 0.144 𝐞₁₃ + -0.0405 𝐞₂₃,
 -0.0517 𝐞₁ + -1.15 𝐞₂ + 0.0548 𝐞₃ + -2.89 𝐞₁₂ + 0.144 𝐞₁₃ + -0.0406 𝐞₂₃,
 1.11e-14 + -0.0518 𝐞₁ + -1.15 𝐞₂ + 0.0548 𝐞₃ + -2.89 𝐞₁₂ + 0.144 𝐞₁₃ + -0.0405 𝐞₂₃)

In [147]:
#5.18
differential(lambda y: differential(f, y, a(x)), x, b), differential(lambda y: differential(f, y, b), x, a(x))

(-0.349 𝐞₁ + -0.1 𝐞₂ + -0.262 𝐞₃, -0.349 𝐞₁ + -0.1 𝐞₂ + -0.262 𝐞₃)

In [148]:
#5.19
tangents = blade_split(I, alg)
frame = multi_frame(tangents)
r_frame = reci_frame(tangents)
# pb_frame = [differential(lambda x: P(v, Ix(x)), x, b) for v in frame]
A1 = f_(PA)(x)
fb = lambda C: differential(fa_(C), x, pb, h=1e-2)
fb_A1 = lambda C: differential(f__(C), x, pb, h=1e-2).sp(A1)
dfb_A1 = derivative(fb_A1, x, alg, frame=frame, r_frame=r_frame, h=1e-2)

(
    dfb_A1 + Pb_(fa_(A1))(x),
    Pb_(fa_(A1))(x),
    fb(A1)
    )

(-0.0352 𝐞₁ + -0.209 𝐞₂ + 0.011 𝐞₃ + 17.7 𝐞₁₂ + -0.964 𝐞₁₃ + 0.358 𝐞₂₃ + -1.22e-19 𝐞₁₂₃,
 -0.000129 𝐞₁ + 0.000161 𝐞₂ + 0.00671 𝐞₃ + 0.000526 𝐞₁₂ + -0.539 𝐞₁₃ + 0.699 𝐞₂₃,
 -0.0352 𝐞₁ + -0.209 𝐞₂ + 0.011 𝐞₃ + 17.7 𝐞₁₂ + -0.963 𝐞₁₃ + 0.358 𝐞₂₃)

In [149]:
#5.20
(
    dfb_A1,
    P(dfb_A1, I),
    P(Pb_(fa_(A1))(x), I),
    P(fb(A1), I)
    )

(-0.035 𝐞₁ + -0.209 𝐞₂ + 0.00434 𝐞₃ + 17.7 𝐞₁₂ + -0.424 𝐞₁₃ + -0.341 𝐞₂₃ + -1.22e-19 𝐞₁₂₃,
 -0.035 𝐞₁ + -0.209 𝐞₂ + 0.00434 𝐞₃ + 17.7 𝐞₁₂ + -0.424 𝐞₁₃ + -0.341 𝐞₂₃,
 5.02e-10 𝐞₁ + -2.39e-08 𝐞₂ + 5.82e-10 𝐞₃ + -1.4e-07 𝐞₁₂ + 3.36e-09 𝐞₁₃ + 2.69e-09 𝐞₂₃,
 -0.035 𝐞₁ + -0.209 𝐞₂ + 0.00434 𝐞₃ + 17.7 𝐞₁₂ + -0.424 𝐞₁₃ + -0.341 𝐞₂₃)

In [150]:
#5.21
C.sp(fb(A1)), fb_A1(C)

(-6.51, -6.51)

In [151]:
#5.22 quadriple differential, so the truncations...
f_vec = lambda a: lambda x: differential(f, x, a, h=1e-3)
fa_ = lambda A: adjoint_outermorphism_(f_vec, A, alg, Ix=Ix, h=1e-3)
fb = lambda b: differential(fa_(A1), x, b, h=1e-3)
P(curl(fb, x, alg, Ix=Ix, h=1e-2), I)

-7.1e-18 𝐞₂ + 1.11e-19 𝐞₃ + -6.13e-06 𝐞₁₂ + 1.47e-07 𝐞₁₃ + 1.18e-07 𝐞₂₃

In [152]:
# Even though these two are the same
f__(PA(x))(x), f__(A(x))(x)

(0.162 + 0.0858 𝐞₁ + 0.0447 𝐞₂ + 0.0329 𝐞₃ + -2.04 𝐞₁₂ + -4.59 𝐞₁₃ + -1.61 𝐞₂₃ + 5.54e-19 𝐞₁₂₃,
 0.162 + 0.0858 𝐞₁ + 0.0447 𝐞₂ + 0.0329 𝐞₃ + -2.04 𝐞₁₂ + -4.59 𝐞₁₃ + -1.61 𝐞₂₃ + 5.54e-19 𝐞₁₂₃)

In [153]:
#5.23 The non-tangent part of A contributes to the exterior derivative

f__ = lambda A: outermorphism_(f_vec, A, alg, Ix=Ix, h=1e-3)
d(lambda x, A: f__(A)(x), x, A(x), alg, Ix, h=1e-2), d(lambda x, A: f__(A)(x), x, PA(x), alg, Ix, h=1e-2)

(-0.139 𝐞₁ + -0.0914 𝐞₂ + -0.0952 𝐞₃ + 0.0523 𝐞₁₂ + 0.117 𝐞₁₃ + 0.0411 𝐞₂₃ + -1e-15 𝐞₁₂₃,
 1.91e-05 𝐞₁ + -3.58e-05 𝐞₂ + 0.000127 𝐞₃ + -1.34e-20 𝐞₁₂ + -3e-20 𝐞₁₃ + -1.05e-20 𝐞₂₃ + -9.99e-16 𝐞₁₂₃)

In [154]:
#5.23 How? The vector part contributes nothing

f__ = lambda A: outermorphism_(f_vec, A, alg, Ix=Ix, h=1e-3)
d(lambda x, A: f__(A)(x), x, a(x), alg, Ix, h=1e-2), d(lambda x, A: f__(A)(x), x, pa(x), alg, Ix, h=1e-2)

(6.61e-19 𝐞₁₂ + 1.59e-18 𝐞₁₃ + 1.85e-18 𝐞₂₃,
 -1.34e-20 𝐞₁₂ + -3e-20 𝐞₁₃ + -1.05e-20 𝐞₂₃)

In [155]:
#5.23 The bivector part contributes
blade = random_r_blade(2, alg)
pblade = P(blade, I)

f__ = lambda A: outermorphism_(f_vec, A, alg, Ix=Ix, h=1e-3)
d(lambda x, A: f__(A)(x), x, blade, alg, Ix, h=1e-2), d(lambda x, A: f__(A)(x), x, pblade, alg, Ix, h=1e-2)

(-0.00883 𝐞₁ + -0.0258 𝐞₂ + -0.0509 𝐞₃ + -3.16e-16 𝐞₁₂₃,
 6.02e-06 𝐞₁ + -1.13e-05 𝐞₂ + 4e-05 𝐞₃ + -3.16e-16 𝐞₁₂₃)

In [156]:
T = lambda x, A: f__(A)(x)
list((inner(blade, r), T(x, inner(blade, r)), o(lambda x: T(x, inner(blade, r)))) for r, o in derivative_gen(x, alg, Ix=nIx, h=1e-3))


[(0.431 𝐞₁ + 0.0125 𝐞₂ + 0.522 𝐞₃,
  0.501 𝐞₁ + -0.079 𝐞₂ + -0.571 𝐞₃ + 5.84e-18 𝐞₁₂₃,
  0.328 𝐞₁ + -0.311 𝐞₂ + -0.393 𝐞₃ + 2.92e-15 𝐞₁₂₃),
 (-0.00214 𝐞₁ + 0.444 𝐞₂ + 0.111 𝐞₃,
  0.374 𝐞₁ + 0.51 𝐞₂ + 0.851 𝐞₃ + 2.7e-21 𝐞₁₂₃,
  -0.337 𝐞₁ + 0.286 𝐞₂ + 0.342 𝐞₃ + 4.09e-15 𝐞₁₂₃)]

In [157]:
T = lambda x, A: f__(A)(x)
list(o(lambda x: T(x, inner(pblade, r))) for r, o in derivative_gen(x, alg, Ix=nIx, h=1e-3))


[0.346 𝐞₁ + -0.287 𝐞₂ + -0.352 𝐞₃ + 2.92e-15 𝐞₁₂₃,
 -0.346 𝐞₁ + 0.287 𝐞₂ + 0.352 𝐞₃ + 4.09e-15 𝐞₁₂₃]

In [158]:
#5.24
d(lambda x, A: P(A, Ix(x)), x, PA(x), alg, Ix)

-1.73e-12 + -2.95e-11 𝐞₁ + 1.09e-10 𝐞₂ + -2.43e-11 𝐞₃ + -9.13e-21 𝐞₁₂ + 2.19e-22 𝐞₁₃ + 1.76e-22 𝐞₂₃

f_ and $fb$ are generalizations of $P$ and $Pb$

Can we generalize the shape operator?

    def shape(Ix, alg):
        return lambda x: lambda A: derivative(lambda x: P(A, Ix(x)), x, alg, Ix=Ix)

A generalization may be:

    def shape_f(f_, Ix, alg):
        return lambda x: lambda A: derivative(f_(A), x, alg, Ix=Ix)

Note that f_(A) is a function of position $x$

In [ ]:
#5.25 Here A' -> C, Ap -> fa_(C), A'p -> P1C, A'r -> C_
fb = lambda C: differential(fa_(C), x, pb, h=1e-3)
Pb_ = lambda A: lambda x: differential(lambda a: P(A(x), Ix(a)), x, pb, h=1e-3)
Pb_1 = lambda A: lambda x: differential(lambda a: P(A(x), f_(Ix)(a)), x, pb, h=1e-3)
P1C = P(C, I1)
C_ = lambda x: C - P1C
(
    fb(C),
    Pb_(fa_(C))(x)+ P(fb(C), I),
    fb(P1C) + fa_(Pb_1(C_)(x))(x)
    )

(-0.0578 𝐞₁ + -1.14 𝐞₂ + 0.0544 𝐞₃ + -2.85 𝐞₁₂ + 0.143 𝐞₁₃ + -0.0413 𝐞₂₃,
 -0.0578 𝐞₁ + -1.14 𝐞₂ + 0.0544 𝐞₃ + -2.85 𝐞₁₂ + 0.143 𝐞₁₃ + -0.0413 𝐞₂₃,
 -0.0578 𝐞₁ + -1.14 𝐞₂ + 0.0544 𝐞₃ + -2.85 𝐞₁₂ + 0.143 𝐞₁₃ + -0.0413 𝐞₂₃)

In [160]:
#5.25b
fb = lambda b: differential(fa_(C), x, b, h=1e-3)
Pb = lambda b: differential(lambda a: P(fa_(C)(x), Ix(a)), x, b, h=1e-3)
(
    curl(fb, x, alg, Ix=Ix, h=1e-3), 
    curl(Pb, x, alg, Ix=Ix, h=1e-3), 
    P(curl(fb, x, alg, Ix=Ix, h=1e-3), I)
    )


(0.000585 𝐞₁₂ + -0.0133 𝐞₁₃ + 0.0534 𝐞₂₃ + -0.287 𝐞₁₂₃,
 -5.55e-11 𝐞₁ + 1.98e-11 𝐞₂ + -1.54e-12 𝐞₃ + 0.000711 𝐞₁₂ + -0.0133 𝐞₁₃ + 0.0535 𝐞₂₃ + -0.287 𝐞₁₂₃,
 -8.88e-19 𝐞₁ + -0.000124 𝐞₁₂ + 2.97e-06 𝐞₁₃ + 2.39e-06 𝐞₂₃)

In [161]:
#5.25c
fb = lambda b: differential(fa_(P1C), x, b, h=1e-3)
Pb_1 = lambda b: differential(lambda a: P(C_(x), f_(Ix)(a)), x, b, h=1e-3)
fa_ = lambda A: adjoint_outermorphism_(f_vec, A, alg, Ix=Ix, h=1e-3)
(
    curl(fb, x, alg, Ix=Ix, h=1e-3), 
    fa_(sum(r^o(Pb_1) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3)))(x),
    )


(0.000567 𝐞₁₂ + -0.0133 𝐞₁₃ + 0.0535 𝐞₂₃ + -0.287 𝐞₁₂₃,
 6.03e-22 𝐞₁ + 2.78e-25 𝐞₂ + 1.16e-23 𝐞₃ + 0.0166 𝐞₁₂ + -0.000397 𝐞₁₃ + -0.000319 𝐞₂₃)

In [164]:
#5.26
# S = lambda A: curl(lambda x: P(A, Ix(x)), x, alg, Ix=Ix, h=1e-3)
S = shape(Ix, alg)(x)
Pb = lambda b: differential(lambda a: P(fa_(C)(x), Ix(a)), x, b, h=1e-3)
fb = lambda C: lambda b: differential(fa_(C), x, b, h=1e-3)
(
    S(fa_(C)(x)), 
    curl(lambda a: P(fa_(C)(x), Ix(a)), x, alg, Ix=Ix, h=1e-3), 
    curl(Pb, x, alg, Ix=Ix, h=1e-3), 
    curl(fb(P1C), x, alg, Ix=Ix, h=1e-3), 
    curl(fb(C), x, alg, Ix=Ix, h=1e-3),
    )

(-2.38e-11 + -3.22e-11 𝐞₁ + 3.13e-10 𝐞₂ + -3.83e-11 𝐞₃ + 0.000711 𝐞₁₂ + -0.0132 𝐞₁₃ + 0.0534 𝐞₂₃ + -0.287 𝐞₁₂₃,
 -1.11e-13 𝐞₁ + -1.98e-14 𝐞₂ + -1.66e-15 𝐞₃ + 0.000711 𝐞₁₂ + -0.0132 𝐞₁₃ + 0.0534 𝐞₂₃ + -0.287 𝐞₁₂₃,
 -5.55e-11 𝐞₁ + 1.98e-11 𝐞₂ + -1.54e-12 𝐞₃ + 0.000711 𝐞₁₂ + -0.0133 𝐞₁₃ + 0.0535 𝐞₂₃ + -0.287 𝐞₁₂₃,
 0.000567 𝐞₁₂ + -0.0133 𝐞₁₃ + 0.0535 𝐞₂₃ + -0.287 𝐞₁₂₃,
 0.000585 𝐞₁₂ + -0.0133 𝐞₁₃ + 0.0534 𝐞₂₃ + -0.287 𝐞₁₂₃)

In [37]:
#5.27
C - P1C, curl(fb(C - P1C), x, alg, Ix=Ix, h=1e-3)

(6.94e-18 + -0.0308 𝐞₁ + -0.121 𝐞₂ + 0.159 𝐞₃ + -0.0785 𝐞₁₂ + 0.297 𝐞₁₃ + 0.758 𝐞₂₃ + 0.962 𝐞₁₂₃,
 -2.59e-05 𝐞₁₂ + 7.18e-07 𝐞₁₃ + 1.74e-06 𝐞₂₃ + -4.26e-06 𝐞₁₂₃)

In [38]:
#5.29 f: M -> N
# Let A be a field on M, also a field on N with a hidden inverse f

def forward_curl(F, x, f, alg, Ix, h=1e-6):
    return sum(r ^ o(F) for r, o in forward_derivative_gen(x, f, alg, Ix, h))

(
    curl(fa(A), x, alg, Ix=Ix, h=1e-3),
    S(fa(A)(x)) +
    fa_(forward_curl(A, x, f, alg, Ix, h=1e-3))(x)
    )

(-0.274 𝐞₁ + 1.64 𝐞₂ + -0.0447 𝐞₃ + 0.917 𝐞₁₂ + -0.103 𝐞₁₃ + 0.0571 𝐞₂₃ + 0.145 𝐞₁₂₃,
 -0.274 𝐞₁ + 1.64 𝐞₂ + -0.0447 𝐞₃ + 0.917 𝐞₁₂ + -0.103 𝐞₁₃ + 0.0571 𝐞₂₃ + 0.145 𝐞₁₂₃)

In [39]:
#5.30 cocurl "commutes" with adjoint with a pushforward
fA = fa(A)
(
    P(curl(fa(A), x, alg, Ix=Ix, h=1e-3), I), 
    fa_(forward_curl(A, x, f, alg, Ix, h=1e-3))(x), 
    sum(fa_(r)(x) ^ P(o(fA), I) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3)),
    sum(fa_(r)(x) ^ o(fA) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3)), # Wrong! Need a projection to keep it intrinsic
    )

(-0.274 𝐞₁ + 1.64 𝐞₂ + -0.0447 𝐞₃ + 0.918 𝐞₁₂ + -0.022 𝐞₁₃ + -0.0177 𝐞₂₃,
 -0.274 𝐞₁ + 1.64 𝐞₂ + -0.0447 𝐞₃ + 0.918 𝐞₁₂ + -0.022 𝐞₁₃ + -0.0177 𝐞₂₃,
 -0.274 𝐞₁ + 1.64 𝐞₂ + -0.0447 𝐞₃ + 0.918 𝐞₁₂ + -0.022 𝐞₁₃ + -0.0177 𝐞₂₃ + 2.78e-17 𝐞₁₂₃,
 -0.274 𝐞₁ + 1.64 𝐞₂ + -0.0447 𝐞₃ + 0.918 𝐞₁₂ + -0.103 𝐞₁₃ + 0.0571 𝐞₂₃ + 0.145 𝐞₁₂₃)

In [40]:
#5.31
nI1 = normalize(I1)
Jf = lambda x: norm(f_(nIx)(x))
fa_(f_(A)(x) * nI1)(x), Jf(x) * PA(x) * nI

(0.785 + 1.6 𝐞₁ + -0.487 𝐞₂ + 0.0424 𝐞₃ + 0.352 𝐞₁₂ + -0.00845 𝐞₁₃ + -0.00678 𝐞₂₃,
 0.785 + 1.6 𝐞₁ + -0.487 𝐞₂ + 0.0424 𝐞₃ + 0.352 𝐞₁₂ + -0.00845 𝐞₁₃ + -0.00678 𝐞₂₃ + -6.94e-18 𝐞₁₂₃)

In [41]:
#5.32a
codiv_ = lambda A: codiv(A, x, alg, Ix=Ix, h=1e-3)
cocurl_ = lambda A: cocurl(A, x, alg, Ix=Ix, h=1e-3)
(
    sum(inner(r, o(f_(PA))) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-2)),
    f__(codiv_(lambda x: Jf(x)*PA(x))/Jf(x))(x),
    f__(codiv_(PA) + inner(cocurl_(lambda x: np.log(Jf(x))),PA(x)))(x)
    )

(3.99 + 0.533 𝐞₁ + -1.31 𝐞₂ + -0.891 𝐞₃ + 6.11e-17 𝐞₁₂ + -7.77e-17 𝐞₁₃ + -4.6e-16 𝐞₂₃,
 3.99 + 0.533 𝐞₁ + -1.31 𝐞₂ + -0.891 𝐞₃ + -8.26e-20 𝐞₁₂ + -6.28e-20 𝐞₁₃ + 1.6e-20 𝐞₂₃ + -1.22e-17 𝐞₁₂₃,
 3.99 + 0.533 𝐞₁ + -1.31 𝐞₂ + -0.891 𝐞₃ + -8.26e-20 𝐞₁₂ + -6.28e-20 𝐞₁₃ + 1.6e-20 𝐞₂₃ + -1.22e-17 𝐞₁₂₃)

In [42]:
#5.32b
div_ = lambda A: div(A, x, alg, Ix=Ix, h=1e-3)
curl_ = lambda A: curl(A, x, alg, Ix=Ix, h=1e-3)
(
    sum(inner(r, o(f_(PA))) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-2)),
    f__(div_(lambda x: Jf(x)*PA(x))/Jf(x))(x),
    f__(div_(PA) + inner(curl_(lambda x: np.log(Jf(x))),PA(x)))(x)
    )

(3.99 + 0.533 𝐞₁ + -1.31 𝐞₂ + -0.891 𝐞₃ + 6.11e-17 𝐞₁₂ + -7.77e-17 𝐞₁₃ + -4.6e-16 𝐞₂₃,
 3.99 + 0.533 𝐞₁ + -1.31 𝐞₂ + -0.891 𝐞₃ + -8.26e-20 𝐞₁₂ + -6.28e-20 𝐞₁₃ + 1.6e-20 𝐞₂₃ + -1.22e-17 𝐞₁₂₃,
 3.99 + 0.533 𝐞₁ + -1.31 𝐞₂ + -0.891 𝐞₃ + -8.26e-20 𝐞₁₂ + -6.28e-20 𝐞₁₃ + 1.6e-20 𝐞₂₃ + -1.22e-17 𝐞₁₂₃)

In [43]:
#5.33
inner(b, f_(A)(x)), f__(inner(fa_(b)(x), A(x)))(x)

(2.13 + 0.826 𝐞₁ + -0.33 𝐞₂ + -0.0909 𝐞₃ + -3.38e-18 𝐞₁₂ + 2.85e-18 𝐞₁₃ + -1.51e-18 𝐞₂₃,
 2.13 + 0.826 𝐞₁ + -0.33 𝐞₂ + -0.0909 𝐞₃ + -1.3e-18 𝐞₁₂ + -9.88e-19 𝐞₁₃ + 2.52e-19 𝐞₂₃ + -7.12e-18 𝐞₁₂₃)

In [44]:
#5.34 codiv "commutes" with outermorphism also with a pushforward but on the LHS
(
    sum(inner(r, f__(o(A))(x)) for r, o in forward_derivative_gen(x, f, alg, Ix)),
    f__(codiv_(A))(x)
    )

(-0.553 + -0.96 𝐞₁ + -0.516 𝐞₂ + -0.578 𝐞₃ + 1.44e-16 𝐞₁₂ + -2.04e-16 𝐞₁₃ + -5.74e-17 𝐞₂₃,
 -0.553 + -0.96 𝐞₁ + -0.516 𝐞₂ + -0.578 𝐞₃ + -5.2e-18 𝐞₁₂ + -3.95e-18 𝐞₁₃ + 1.01e-18 𝐞₂₃ + 2.01e-18 𝐞₁₂₃)

In [45]:
#5.35
def forward_curl(F, x, f, alg, Ix, h=1e-3):
    return sum(r ^ o(F) for r, o in forward_derivative_gen(x, f, alg, Ix, h=h))

def forward_div(F, x, f, alg, Ix, h=1e-3):
    return sum(inner(r, o(F)) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3))
(
    forward_div(f__(PA(x)), x, f, alg, Ix),
    f__(inner(curl_(lambda x: np.log(Jf(x))), A(x)))(x),
    inner(forward_curl(lambda x: np.log(Jf(x)), x, f, alg, Ix), f_(A)(x)),
)

(4.55 + 1.55 𝐞₁ + -0.813 𝐞₂ + -0.318 𝐞₃ + -3.88e-15 𝐞₁₂ + 9.71e-15 𝐞₁₃ + 1.76e-14 𝐞₂₃,
 4.55 + 1.55 𝐞₁ + -0.814 𝐞₂ + -0.318 𝐞₃ + -1.47e-17 𝐞₁₂₃,
 4.55 + 1.55 𝐞₁ + -0.813 𝐞₂ + -0.318 𝐞₃ + -3.62e-16 𝐞₁₂ + 3.19e-16 𝐞₁₃ + -6.21e-16 𝐞₂₃)

In [46]:
#5.36 simplify to differential only when A is a vector
(   
    differential(G, f(x), f_(A)(x), h=1e-3), 
    differential(G, f(x), f_(a)(x), h=1e-3), 
    sum(f__(alg.sp(r, a(x)))(x) * o(F) for r, o in derivative_gen(x, alg, Ix=Ix, h=1e-3)),
    sum(f__(inner(A(x), r))(x) * o(F) for r, o in derivative_gen(x, alg, Ix=Ix, h=1e-3))
    )

(6.39 + 3.03 𝐞₁ + 1.76 𝐞₂ + 3.0 𝐞₃ + 1.45 𝐞₁₂ + -1.38 𝐞₁₃ + 3.55 𝐞₂₃ + 4.97 𝐞₁₂₃,
 4.04 + 2.79 𝐞₁ + 1.41 𝐞₂ + 1.61 𝐞₃ + 1.42 𝐞₁₂ + -1.25 𝐞₁₃ + 2.46 𝐞₂₃ + 3.85 𝐞₁₂₃,
 4.04 + 2.79 𝐞₁ + 1.41 𝐞₂ + 1.61 𝐞₃ + 1.42 𝐞₁₂ + -1.25 𝐞₁₃ + 2.46 𝐞₂₃ + 3.85 𝐞₁₂₃,
 4.04 + 0.829 𝐞₁ + 0.695 𝐞₂ + 3.48 𝐞₃ + 0.391 𝐞₁₂ + -4.01 𝐞₁₃ + 2.67 𝐞₂₃ + 3.85 𝐞₁₂₃)

In [47]:
#5.36
(
    sum(inner(f__(A(x))(x), r) * o(G) for r, o in f_derivative_gen(x, f, alg, Ix, h=1e-3)),
    sum(inner(f__(A(x))(x), r) * o(F) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3)),
    sum(f__(inner(A(x), r))(x) * o(F) for r, o in derivative_gen(x, alg, Ix=Ix, h=1e-3))
    )

(4.04 + 0.829 𝐞₁ + 0.695 𝐞₂ + 3.48 𝐞₃ + 0.391 𝐞₁₂ + -4.01 𝐞₁₃ + 2.67 𝐞₂₃ + 3.85 𝐞₁₂₃,
 4.04 + 0.829 𝐞₁ + 0.694 𝐞₂ + 3.48 𝐞₃ + 0.39 𝐞₁₂ + -4.0 𝐞₁₃ + 2.67 𝐞₂₃ + 3.85 𝐞₁₂₃,
 4.04 + 0.829 𝐞₁ + 0.695 𝐞₂ + 3.48 𝐞₃ + 0.391 𝐞₁₂ + -4.01 𝐞₁₃ + 2.67 𝐞₂₃ + 3.85 𝐞₁₂₃)

In [48]:
#5.37 holds for tangent fields
B = lambda x: (C*x+D)**2
def forward_lie_bracket(A, B, f, alg, Ix, h=1e-6):
    return lambda x: sum((inner(A(x),r)^o(B)) - (o(A)^inner(r,B(x))) for r, o in forward_derivative_gen(x, f, alg, Ix, h))

PB = lambda x: P(B(x), Ix(x))

(
    f_(lie_bracket(A, B, alg, Ix))(x), 
    forward_lie_bracket(f_(A), f_(B), f, alg, Ix, h=1e-3)(x), 
    f_(lie_bracket(PA, PB, alg, Ix))(x), 
    forward_lie_bracket(f_(PA), f_(PB), f, alg, Ix, h=1e-3)(x)
    )

(-1.5 + -2.91 𝐞₁ + -2.37 𝐞₂ + -2.37 𝐞₃ + 8.29 𝐞₁₂ + 6.29 𝐞₁₃ + -1.6 𝐞₂₃ + 4.47e-19 𝐞₁₂₃,
 -1.49 + -2.89 𝐞₁ + -2.36 𝐞₂ + -2.35 𝐞₃ + 8.24 𝐞₁₂ + 6.26 𝐞₁₃ + -1.59 𝐞₂₃ + 3.75e-05 𝐞₁₂₃,
 -1.5 + -2.89 𝐞₁ + -2.37 𝐞₂ + -2.36 𝐞₃ + 8.24 𝐞₁₂ + 6.26 𝐞₁₃ + -1.59 𝐞₂₃ + 3.67e-19 𝐞₁₂₃,
 -1.49 + -2.89 𝐞₁ + -2.36 𝐞₂ + -2.35 𝐞₃ + 8.24 𝐞₁₂ + 6.26 𝐞₁₃ + -1.59 𝐞₂₃ + 3.75e-05 𝐞₁₂₃)

In [49]:
#5.39
r = 1
s = 2
P1B = lambda x: P(B(x), f_(Ix)(x))
Ar = lambda x: PA(x).grade(r)
Bs = lambda x: P1B(x).grade(s)

def forward_dual_bracket(A, B, f, alg, nIx, h=1e-6):
    # nIx: normalized pseudoscalar
    f_vec = lambda a: lambda x: differential(f, x, a)
    f_ = lambda A: outermorphism(f_vec, A, alg, Ix=Ix)
    nIx = lambda x: normalize(Ix(x))
    return lambda x: forward_lie_bracket(A, lambda x: B(x)*f_(nIx)(x), f, alg, nIx, h)(x) * inv(f_(nIx)(x))
(
    fa(forward_dual_bracket(f_(Ar), Bs, f, alg, nIx, h=1e-3))(x),
    dual_bracket(Ar, fa(Bs), alg, nIx, h=1e-3)(x) -
    inner(inner(Ar(x), cocurl_(lambda x: np.log(Jf(x)))), fa(Bs)(x))
)

(-0.000152 + 5.04 𝐞₁₂ + -0.121 𝐞₁₃ + -0.0971 𝐞₂₃,
 4.99e-17 + 5.67 𝐞₁₂ + -0.136 𝐞₁₃ + -0.109 𝐞₂₃)

In [50]:
#6.1
s = lambda x: ((C + D*x) ** 2)[0]
f = lambda x: s(x)*x
s(x), terms_ratio(x, f(x)), terms_ratio(Ix(x), f_(Ix)(x))

(-0.9674202003069668,
 array([-1.03367699, -1.03367699, -1.03367699]),
 array([ 0.97743069,  0.13460986, -0.03388521]))

In [51]:
#6.2
r = 2
Ar = lambda x: A(x).grade(r)
PAr = P(Ar(x), I)
(
    f_(Ar)(x), 
    (s(x) ** r * PAr) + (s(x)**(r-1) * sum((PAr|r) * o(s) for r, o in derivative_gen(x, alg, Ix=Ix)) ^ x),
    s(x)** r * (PAr + (sum((PAr|r) * o(lambda x: np.log(abs(s(x)))) for r, o in derivative_gen(x, alg, Ix=Ix)) ^ x))
    )

(-1.21 𝐞₁₂ + 0.21 𝐞₁₃ + -0.669 𝐞₂₃,
 -1.21 𝐞₁₂ + 0.21 𝐞₁₃ + -0.669 𝐞₂₃,
 -1.21 𝐞₁₂ + 0.21 𝐞₁₃ + -0.669 𝐞₂₃)

In [52]:
#6.2 on Ix
(
    f_(Ix)(x), 
    (s(x) ** r * I) + (s(x)**(r-1) * sum((I|r) * o(s) for r, o in derivative_gen(x, alg, Ix=Ix)) ^ x),
    (s(x) ** r * I), (s(x)**(r-1) * sum((I|r) * o(s) for r, o in derivative_gen(x, alg, Ix=Ix)) ^ x),
    )

(2.86 𝐞₁₂ + -0.498 𝐞₁₃ + 1.59 𝐞₂₃,
 2.86 𝐞₁₂ + -0.498 𝐞₁₃ + 1.59 𝐞₂₃,
 2.62 𝐞₁₂ + -0.0627 𝐞₁₃ + -0.0504 𝐞₂₃,
 0.244 𝐞₁₂ + -0.435 𝐞₁₃ + 1.64 𝐞₂₃)

In [53]:
#6.3
(
    fa(Ar)(x), 
    (s(x) ** r * PAr) + (s(x)**(r-1) * derivative(s, x, alg, Ix=Ix) ^ P(x|Ar(x), I)),
    )

(-1.39 𝐞₁₂ + 0.0333 𝐞₁₃ + 0.0267 𝐞₂₃, -1.39 𝐞₁₂ + 0.0333 𝐞₁₃ + 0.0267 𝐞₂₃)

In [54]:
#6.4 central projection (normalization)
s = lambda x: 1/norm(x) # so f(x) = normalize(x)
I1x = lambda x: f(x) * e123
n = lambda x: -e123 * nIx(x)
nI, n(x) * e123, e123*n(x)

(1.0 𝐞₁₂ + -0.024 𝐞₁₃ + -0.0192 𝐞₂₃,
 1.0 𝐞₁₂ + -0.024 𝐞₁₃ + -0.0192 𝐞₂₃,
 1.0 𝐞₁₂ + -0.024 𝐞₁₃ + -0.0192 𝐞₂₃)

In [55]:
#6.5a The Jf in our example indicates an opposite orientation
f_(nIx)(x), Jf(x)*I1x(x), (s(x) ** 2 * nI ) + ((s(x)*nI*derivative(s, x, alg, Ix=Ix)) ^ x)

(0.492 𝐞₁₂ + -0.295 𝐞₁₃ + -0.152 𝐞₂₃,
 -0.492 𝐞₁₂ + 0.295 𝐞₁₃ + 0.152 𝐞₂₃,
 0.492 𝐞₁₂ + -0.295 𝐞₁₃ + -0.152 𝐞₂₃)

In [56]:
#6.5b
f_(nIx)(x)*e123.reverse(), Jf(x)*normalize(x), (s(x) ** 2 * n(x) ) + ((s(x)*n(x)^derivative(s, x, alg, Ix=Ix)) | x)

(-0.152 𝐞₁ + 0.295 𝐞₂ + 0.492 𝐞₃,
 0.152 𝐞₁ + -0.295 𝐞₂ + -0.492 𝐞₃,
 -0.152 𝐞₁ + 0.295 𝐞₂ + 0.492 𝐞₃)

In [57]:
#6.6 The f-inverse from S to M is just x -> x/s(x)
x_ = f(x)

n(x)/Jf(x), (s(x) ** (-2) * x_ ) + ((x_^forward_curl(lambda x: 1/s(x), x, f, alg, Ix=Ix)) | x_)/s(x)

(-0.0324 𝐞₁ + 0.0404 𝐞₂ + 1.69 𝐞₃, 0.0324 𝐞₁ + -0.0405 𝐞₂ + -1.69 𝐞₃)

In [58]:
#6.7 terms in the RHS of 6.6 are perpendicular, so square and sum
fc = forward_curl(lambda x: 1/s(x), x, f, alg, Ix=Ix)
Jf(x), 1/np.sqrt(((s(x) ** (-4)) + (s(x)**(-2)*fc**2))[0]), s(x) ** 2 * n(x) | x_

(0.5931314802712396, 0.593153293723961, -0.593)

In [59]:
#6.9 inversion
s = lambda x: 1/(x**2)[0]
Ar = lambda x: PA(x).grade(r)
f_(Ar)(x), fa(Ar)(x) # not equal for all manifolds

(-0.234 𝐞₁₂ + 0.474 𝐞₁₃ + 0.239 𝐞₂₃, -0.249 𝐞₁₂ + 0.00598 𝐞₁₃ + 0.0048 𝐞₂₃)

In [60]:
# But on Euclidean space
Ix = lambda x: e123
f_(Ar)(x), fa(Ar)(x)

(0.787 𝐞₁₂ + 0.914 𝐞₁₃ + 0.364 𝐞₂₃, 0.787 𝐞₁₂ + 0.914 𝐞₁₃ + 0.364 𝐞₂₃)

In [61]:
# inversion is curl free
curl(f, x, alg, grade=1)

1.39e-11 𝐞₁₂ + -1.39e-11 𝐞₁₃ + 5.55e-11 𝐞₂₃

In [62]:
#6.10
def forward_derivative(F, x, f, alg, Ix, h=1e-3):
    return sum(r * o(F) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3))

(
    derivative(F, x, alg, grade=1), 
    sum(-x.sw(r)/(x**4)[0] * o(F) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3)),
    forward_derivative(F, x, f, alg, Ix),
    sum(-x.sw(r) * o(F) for r, o in derivative_gen(x, alg, Ix=Ix, h=1e-3))
    )

(1.11 + 0.122 𝐞₁ + 0.314 𝐞₂ + 0.384 𝐞₃ + 0.673 𝐞₁₂ + 1.21 𝐞₁₃ + 1.38 𝐞₂₃ + 0.983 𝐞₁₂₃,
 1.11 + 0.122 𝐞₁ + 0.314 𝐞₂ + 0.384 𝐞₃ + 0.673 𝐞₁₂ + 1.21 𝐞₁₃ + 1.38 𝐞₂₃ + 0.983 𝐞₁₂₃,
 4.77 + 0.497 𝐞₁ + -0.181 𝐞₂ + -0.501 𝐞₃ + 1.9 𝐞₁₂ + 1.16 𝐞₁₃ + 1.68 𝐞₂₃ + 4.21 𝐞₁₂₃,
 4.77 + 0.497 𝐞₁ + -0.181 𝐞₂ + -0.501 𝐞₃ + 1.9 𝐞₁₂ + 1.16 𝐞₁₃ + 1.68 𝐞₂₃ + 4.21 𝐞₁₂₃)

In [63]:
#6.11 It's wrong because the derivative measures several nearby points around x, 
# but not directly on x, so no direct cancellation of x
deriv_ = lambda F: lambda x: derivative(F, x, alg, Ix=Ix, h=1e-3)
f_deriv_ = lambda F: lambda x: sum(r/(x**4)[0] * o(F) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3))
forward_deriv_ = lambda F: lambda x: sum(r * o(F) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3))
ff_deriv_ = lambda F: lambda x: sum(-x.sw(r)/(x**4)[0] * o(F) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3))

(
    deriv_(deriv_(F))(x), 
    f_deriv_(f_deriv_(F))(x), 
    forward_deriv_(forward_deriv_(F))(x)/(x**4)[0], 
    ff_deriv_(ff_deriv_(F))(x)
    )

(2.21 + -0.477 𝐞₁ + 0.928 𝐞₂ + 1.55 𝐞₃ + 1.37 𝐞₁₂ + -0.819 𝐞₁₃ + -0.421 𝐞₂₃ + 0.664 𝐞₁₂₃,
 2.28 + 3.73 𝐞₁ + -0.633 𝐞₂ + -5.19 𝐞₃ + -3.8 𝐞₁₂ + 2.79 𝐞₁₃ + 1.44 𝐞₂₃ + -0.788 𝐞₁₂₃,
 3.14 + 0.25 𝐞₁ + -0.138 𝐞₂ + 0.0156 𝐞₃ + -0.00545 𝐞₁₂ + 0.0943 𝐞₁₃ + 0.162 𝐞₂₃ + -0.275 𝐞₁₂₃,
 2.31 + -0.718 𝐞₁ + 1.01 𝐞₂ + 1.35 𝐞₃ + 1.23 𝐞₁₂ + -0.876 𝐞₁₃ + -0.581 𝐞₂₃ + 0.484 𝐞₁₂₃)

In [64]:
#6.12
Jf(x), -e123*f__(e123)(x), -1/(x**6)[0]

(0.34512491747611357, -0.345, -0.34512494149178446)

In [65]:
def Ix(x):
    p = np.arctan((4/5)*x.e2/x.e1)
    t = np.arccos(x.e3)
    return e_p(p, t) ^ e_t(p, t)

In [66]:
#6.14
n = normalize(random_r_blade(1, alg))
s = lambda x: ((C + D*x) ** 2)[0]
f = lambda x: x + n*s(x)
ds = derivative(s, x, alg, Ix=Ix)
f_(A)(x), PA(x) + (inner(PA(x),ds) ^ n)

(0.529 + -0.188 𝐞₁ + -1.8 𝐞₂ + 0.164 𝐞₃ + -0.57 𝐞₁₂ + 0.0419 𝐞₁₃ + -0.0969 𝐞₂₃ + -2.54e-18 𝐞₁₂₃,
 0.529 + -0.188 𝐞₁ + -1.8 𝐞₂ + 0.164 𝐞₃ + -0.57 𝐞₁₂ + 0.0419 𝐞₁₃ + -0.0969 𝐞₂₃ + 4.66e-18 𝐞₁₂₃)

In [67]:
#6.15
fa(A)(x), P(A(x) + (ds ^ inner(n, A(x))), I)

(0.529 + 0.615 𝐞₁ + -2.04 𝐞₂ + 0.0609 𝐞₃ + -0.598 𝐞₁₂ + 0.0144 𝐞₁₃ + 0.0115 𝐞₂₃,
 0.529 + 0.615 𝐞₁ + -2.04 𝐞₂ + 0.0609 𝐞₃ + -0.598 𝐞₁₂ + 0.0144 𝐞₁₃ + 0.0115 𝐞₂₃)

In [68]:
#6.16 because s is scalar, ds is tangent
Pn = P(n,I)
Kf = nI.reverse() * f__(nI)(x)
Kf, 1 + differential(s, x, Pn) + (ds^(n - Pn))


(0.483 + -0.00265 𝐞₁₂ + -0.0915 𝐞₁₃ + -0.0239 𝐞₂₃,
 0.483 + -0.00265 𝐞₁₂ + -0.0915 𝐞₁₃ + -0.0239 𝐞₂₃)

In [69]:
#6.17 Kf**2 = (Ir * f_(I))r * Ir * f_(I) = f_(I)r * f_(I) = Jf**2
normsq(Kf), Jf(x) ** 2, (1 + differential(s, x, Pn))**2 + ds**2 * normsq(nI ^n)

(0.2421327289029572,
 0.24213217590918257,
 0.242 + 7.53e-22 𝐞₁₂ + -7.6e-21 𝐞₁₃ + 5.91e-20 𝐞₂₃)

In [70]:
#6.18
(
    deriv_(F)(x), 
    sum(fa_(r)(x) * o(F) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3)),
    sum(P(r + ds * (n|r), I) * o(F) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3)),
    )

(2.36 + 0.611 𝐞₁ + -1.71 𝐞₂ + 0.519 𝐞₃ + -0.474 𝐞₁₂ + 1.16 𝐞₁₃ + 1.36 𝐞₂₃ + 2.08 𝐞₁₂₃,
 2.36 + 0.611 𝐞₁ + -1.71 𝐞₂ + 0.52 𝐞₃ + -0.475 𝐞₁₂ + 1.16 𝐞₁₃ + 1.36 𝐞₂₃ + 2.08 𝐞₁₂₃,
 2.36 + 0.611 𝐞₁ + -1.71 𝐞₂ + 0.52 𝐞₃ + -0.475 𝐞₁₂ + 1.16 𝐞₁₃ + 1.36 𝐞₂₃ + 2.08 𝐞₁₂₃)

In [71]:
#6.19
Ix = lambda x: n * e123
I = Ix(x)
v = random_r_blade(1, alg)
x = v - P(v, n)
x|n, P(n, I)

(3.82e-17, )

In [72]:
#6.20b
I1 = f_(Ix)(x)
nI = nIx(x)
nI1 = normalize(I1)
n1 = -nI1 * e123
nI1, n1*e123

(-0.0372 𝐞₁₂ + 0.483 𝐞₁₃ + 0.875 𝐞₂₃, -0.0372 𝐞₁₂ + 0.483 𝐞₁₃ + 0.875 𝐞₂₃)

In [73]:
#6.21
ds = derivative(s, x, alg, Ix=Ix)
f_(A)(x), PA(x) + (inner(PA(x),ds) ^ n), PA(x) + (inner(PA(x),ds) * n)

(0.269 + -0.474 𝐞₁ + -0.877 𝐞₂ + 0.233 𝐞₃ + 0.0579 𝐞₁₂ + -0.751 𝐞₁₃ + -1.36 𝐞₂₃ + 2.56e-17 𝐞₁₂₃,
 0.269 + -0.474 𝐞₁ + -0.877 𝐞₂ + 0.233 𝐞₃ + 0.0579 𝐞₁₂ + -0.751 𝐞₁₃ + -1.36 𝐞₂₃,
 0.269 + -0.474 𝐞₁ + -0.877 𝐞₂ + 0.233 𝐞₃ + 0.0579 𝐞₁₂ + -0.751 𝐞₁₃ + -1.36 𝐞₂₃)

In [74]:
nIx(x), f_(nIx)(x), nI * (1 + (ds ^ n)), Jf(x)*nI1

(0.147 𝐞₁₂ + -0.731 𝐞₁₃ + 0.666 𝐞₂₃,
 -0.165 𝐞₁₂ + 2.15 𝐞₁₃ + 3.89 𝐞₂₃,
 -2.78e-17 + -0.165 𝐞₁₂ + 2.15 𝐞₁₃ + 3.89 𝐞₂₃,
 -0.165 𝐞₁₂ + 2.15 𝐞₁₃ + 3.89 𝐞₂₃)

In [75]:
#6.22
-nI * f_(nIx)(x), Jf(x) * n * n1, 1 + ds*n

(1.0 + -4.27 𝐞₁₂ + -0.684 𝐞₁₃ + 0.195 𝐞₂₃,
 1.0 + -4.27 𝐞₁₂ + -0.684 𝐞₁₃ + 0.195 𝐞₂₃,
 1.0 + -4.27 𝐞₁₂ + -0.684 𝐞₁₃ + 0.195 𝐞₂₃)

In [76]:
#6.23
n1, (n - ds)/Jf(x)

(0.875 𝐞₁ + -0.483 𝐞₂ + -0.0372 𝐞₃,
 0.875 𝐞₁ + -0.483 𝐞₂ + -0.0372 𝐞₃ + -4.88e-18 𝐞₁₂₃)

In [77]:
#6.24
Jf(x), norm(n - ds)

(4.44465318579657, 4.444653106457838)

In [78]:
#6.25
derivative(F, x, alg, Ix=Ix), sum((P(r, I)+ds*(n|r))*o(F) for r, o in forward_derivative_gen(x, f, alg, Ix))

(3.18 + 1.64 𝐞₁ + -2.82 𝐞₂ + 6.58 𝐞₃ + -4.8 𝐞₁₂ + -1.12 𝐞₁₃ + -0.164 𝐞₂₃ + 2.8 𝐞₁₂₃,
 3.18 + 1.64 𝐞₁ + -2.82 𝐞₂ + 6.58 𝐞₃ + -4.8 𝐞₁₂ + -1.12 𝐞₁₃ + -0.164 𝐞₂₃ + 2.8 𝐞₁₂₃)

In [79]:
#6.26
s(x), n | f(x)

(-0.7230207086740077, -0.723)

In [80]:
#6.27
x1 = f(x)
f(P(x1, I)), x1

(-0.619 𝐞₁ + -0.408 𝐞₂ + -0.0848 𝐞₃, -0.619 𝐞₁ + -0.408 𝐞₂ + -0.0848 𝐞₃)

In [81]:
#6.28
g = lambda x: P(x, I)
g_vec = lambda a: lambda x: differential(g, x, a)
g_ = lambda A: outermorphism(g_vec, A, alg, Ix=f_(Ix))
P1A = lambda x: P(A(x), I1)
g_(P1A)(x), P(P1A(x), I)

(0.269 + -0.0498 𝐞₁ + -0.0327 𝐞₂ + 0.387 𝐞₃ + 0.00376 𝐞₁₂ + -0.0186 𝐞₁₃ + 0.017 𝐞₂₃,
 0.269 + -0.0498 𝐞₁ + -0.0327 𝐞₂ + 0.387 𝐞₃ + 0.00376 𝐞₁₂ + -0.0186 𝐞₁₃ + 0.017 𝐞₂₃)

In [82]:
#6.29
ga = lambda A: adjoint_outermorphism(g_vec, A, alg, Ix=f_(Ix))
ga(A)(x), P(PA(x), I1)

(0.269 + -0.0407 𝐞₁ + -0.103 𝐞₂ + 0.376 𝐞₃ + 0.00293 𝐞₁₂ + -0.038 𝐞₁₃ + -0.0689 𝐞₂₃,
 0.269 + -0.0407 𝐞₁ + -0.103 𝐞₂ + 0.376 𝐞₃ + 0.00293 𝐞₁₂ + -0.038 𝐞₁₃ + -0.0689 𝐞₂₃)

In [83]:
#6.30
forward_deriv_(F)(x), sum(P(r, I1)*o(F) for r, o in derivative_gen(x, alg, Ix=Ix))

(3.18 + -0.319 𝐞₁ + -0.632 𝐞₂ + 0.682 𝐞₃ + 1.97 𝐞₁₂ + 0.212 𝐞₁₃ + -0.0333 𝐞₂₃ + 2.8 𝐞₁₂₃,
 3.18 + -0.32 𝐞₁ + -0.632 𝐞₂ + 0.682 𝐞₃ + 1.97 𝐞₁₂ + 0.212 𝐞₁₃ + -0.0331 𝐞₂₃ + 2.8 𝐞₁₂₃)

In [84]:
#6.31
Ix = lambda x: n * e123
l = lambda x: 0.5 * normsq(x-n)
f = lambda x: n + ((x-n)/l(x))
f(x), -(x-n).sw(n)/normsq(x-n)

(-0.888 𝐞₁ + -0.449 𝐞₂ + -0.0956 𝐞₃, -0.888 𝐞₁ + -0.449 𝐞₂ + -0.0956 𝐞₃)

In [85]:
#6.34
f_(pa)(x), -(x-n).sw(pa(x))/normsq(x-n)/l(x)

(0.165 𝐞₁ + -0.474 𝐞₂ + 0.696 𝐞₃ + 4.95e-17 𝐞₁₂₃,
 0.165 𝐞₁ + -0.474 𝐞₂ + 0.696 𝐞₃)

In [86]:
#6.35
pb = P(b, I)
f_(pa)(x) * f__(pb)(x), (x-n).sw(pa(x)*pb)/normsq(x-n)/l(x)**2

(0.815 + -0.0879 𝐞₁₂ + 0.413 𝐞₁₃ + -0.816 𝐞₂₃,
 0.815 + -0.0879 𝐞₁₂ + 0.413 𝐞₁₃ + -0.816 𝐞₂₃)

In [87]:
#6.36
pa(x) | pb / l(x)**2

0.815

In [88]:
#6.37
f__(pa(x) ^ pb)(x)

-0.0879 𝐞₁₂ + 0.413 𝐞₁₃ + -0.816 𝐞₂₃

In [89]:
#6.38
r=2
f_(Ar)(x), (x-n).sw(Ar(x))/normsq(x-n)/(-l(x))**r

(-0.125 𝐞₁₂ + 0.589 𝐞₁₃ + -1.16 𝐞₂₃, -0.125 𝐞₁₂ + 0.589 𝐞₁₃ + -1.16 𝐞₂₃)

In [90]:
#6.39
s = 1
PB = lambda x: P(B(x), Ix(x))
Bs = lambda x: PB(x).grade(s)
f_(Ar)(x) * f_(Bs)(x), (x-n).sw(Ar(x)*Bs(x))/normsq(x-n)/(-l(x))**(r+s)

(-0.676 𝐞₁ + 0.672 𝐞₂ + 3.13 𝐞₃ + 2.22e-16 𝐞₁₂₃,
 -0.676 𝐞₁ + 0.672 𝐞₂ + 3.13 𝐞₃ + 2.01e-16 𝐞₁₂₃)

In [91]:
#6.41
A1r = f_(Ar)
fa(A1r)(x), (x-n).sw(A1r(x))/normsq(x-n)/(-l(x))**r, f_(A1r)(x),

(-0.723 𝐞₁₂ + 3.58 𝐞₁₃ + -3.27 𝐞₂₃,
 -0.723 𝐞₁₂ + 3.58 𝐞₁₃ + -3.27 𝐞₂₃,
 -0.438 𝐞₁₂ + 2.06 𝐞₁₃ + -4.07 𝐞₂₃)